<h3>Auto-tagger v1</h3>

This script attempts to iterate through a list of ADLS storage assets and apply (managed) attributes against each asset in order to make them more discoverable and improve the search ranking when adding attribute terms to search term.
The business logic iterates through assets and extracts folder names to a pre-defined depth in the folder hierarchy. Folder names can then be compared against your own key value pairs, to extract the business value. This is a common approach for a data lake structure based on a predefined taxonomy where folder names are acronyms (the keys).
Currently this script only supports ADLS assets and two folder levels deep, in this example which is client and market.

<small>MIT License

Copyright (c) 2022 Nick Hurt

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom  he Software is furnished to do so, subject to the following conditions:  The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</small>

<h4>Option 1: Load lookup values from json file into dictionary object</h4>

In [ ]:
import json
with open("clients.json", encoding='utf-8') as f:
  #format should be {"code":"A_AAIR","name":"AA Ireland"},{"code":"A_ABBO","name":"Abbott"},{"code":"A_ACFH","name":"Abercrombie and Fitch"}...{}
  clientjson = json.load(f)
clientdic = dict()
for client in clientjson:
    #print(client)
    clientdic[client['code']] = client['name']

#print(list(clientdic))


with open("markets.json", encoding='utf-8') as f:
  #format should be {"aliasName":"Afghanistan","code":"AF","threeLetterCode":"AFG","name":"Afghanistan"},{"aliasName":"Albania","code":"AL","threeLetterCode":"ALB","name":"Albania"},...{}
  marketjson = json.load(f)
marketdic = dict()
for market in marketjson:
    #print(client)
    marketdic[market['code']] = market['name']

#print(list(marketdic))

<h4>Option 2: Load lookup values from SQL database table(s) into dictionary object</h4>
Simple boilerplate code to populate a dictionary object from two columns from SQL table.  Replace the [tokens] with your details

In [ ]:
import pyodbc
# Replace the [tokens] with your details
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=tcp:[azuresqlsrvname].database.windows.net,1433;Database=[databasename]];Uid=[username];Pwd=[password];Encrypt=yes;TrustServerCertificate=no;Connection Timeout=10;")
cursor = cnxn.cursor()
clientdic = dict()

try:
    fetchsql = """select client_key, client_value from lookuptable """
    cursor.execute(fetchsql)
    row = cursor.fetchone()
    while row:
         clientdic[row[0]] = row[1]
         row = cursor.fetchone()

except pyodbc.DatabaseError as err:
    #cnxn.commit()
    sqlstate = err.args[1]
    sqlstate = sqlstate.split(".")
    print('Warning occured while trying to populate diction from sql table. Error message: '.join(sqlstate))
    


<h4> Obtain auth token to interact with the Purview (Atlas) APIs</h4>
Fill the variable values below.
Requires service principal and secret.

In [ ]:
#pip install requests

import requests
import json
# Client credentials + Purview details (See https://docs.microsoft.com/en-us/azure/purview/tutorial-using-rest-apis)

purview_account_name = ""
client_id = ""
client_secret = ""
resource = "https://purview.azure.net"
tenant_id = ""


# oauth2 login
url = "https://login.microsoftonline.com/" + tenant_id + "/oauth2/token"

# Login and get token
payload='grant_type=client_credentials&client_id=' + client_id + '&client_secret=' + client_secret + '&resource=' + resource
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
}

response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)

# Set the access token
access_token = response['access_token']


<h4>Create the attribute definition</h4>
See https://docs.microsoft.com/en-us/azure/purview/tutorial-atlas-2-2-apis

In [ ]:
purview_endpoint = "https://"+purview_account_name+".purview.azure.com" 
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}
tagName=''
if tagName == '':
  url = purview_endpoint + "/catalog/api/atlas/v2/types/typedefs"
  print(url)
  body = json.dumps({"businessMetadataDefs":[{"category":"BUSINESS_METADATA","createdBy":"admin","updatedBy":"admin","version":1,"typeVersion":"1.1","name":"Data Product","description":"","attributeDefs":[{"name":"Client","typeName":"string","isOptional":True,"cardinality":"SINGLE","isUnique":False,"isIndexable":True,"options":{"maxStrLength":"500","applicableEntityTypes":"[\"Referenceable\"]"}},{"name":"Market","typeName":"string","isOptional":True,"cardinality":"SINGLE","isUnique":False,"isIndexable":True,"options":{"maxStrLength":"500","applicableEntityTypes":"[\"Referenceable\"]"}}]}]})

print(body)
response = json.loads(requests.request("POST", url, headers=headers, data=body).content)
print(response)

<h4>Search for assets to apply (managed) attributes against.</h4> 
Currently there is no way to do this in bulk so it may take a significant amount of time depending on the number of assets.
Note there is also a limit of 100,000 assets that can be returned from a search so it may be best to try to do this per collection.
If the catalog contains more than 100,000 assets (excluding glossary terms), then the search is performed in federated manner using "facets". "facet_type" is configurable. Based on the best possible distrubtion of assets (to be under 100,000 by sub type) choose the facet_type (e.g. entityType, assetTpye, collectionId). If the sub-types still have more than 100,000 assets then only upto 100,000 assets will be exported for those sub types.

In [ ]:
purview_endpoint = "https://"+purview_account_name+".catalog.purview.azure.com" 
url = purview_endpoint + "/api/search/query?api-version=2021-05-01-preview"

# Search for assets to apply managed attributes to. Apply filters as necessary, here are some examples to add to the payload.
# 
# eg "filter": {"entityType": "azure_datalake_gen2_resource_set"},
#    "filter": {"entityType": "azure_datalake_gen2_path"},
#    "filter": {"collectionId": "myCollectionName"},
payload = json.dumps({
  "keywords": "https://somedatalake.dfs.core.windows.net/gold/*",
  "filter":{"collectionId": "PurviewDemo1","entityType": "azure_datalake_gen2_resource_set"},
  "limit": 100
})
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)
print('number of assets found: ' + str(response['@search.count']))
noassets = len(response['value'])
#print(noassets)

purviewresults = response['value']
#print(purviewresults)


<h4>Iterate through the results and depending on the folder depth will use different lookup values</h4>


In [ ]:
for a in range(0,noassets):
  adlpath = purviewresults[a]['qualifiedName']
  #print(adlpath)
  pathstr = adlpath.replace("https://","")
  pathparts = pathstr.split("/",-1)
  i=0
  tagName=''
  for folder in pathparts:
    if i!=0 and i<=2:
      #print("level " + str(i) + " token : " +folder)
      if i == 1: # client lookup
        # perform any lightweight cleansing/data corrections
        cleanfolder = folder.upper()#.replace('-','_').replace('OUR','OURW')
        try:
          clienttag = clientdic[cleanfolder]
        except KeyError:
          clienttag = ''

      if i == 2: # market look up
        # perform any lightweight cleansing/data corrections
        cleanfolder = folder.upper()#.replace('UK','GB').replace("’","'")
        try:
          markettag = marketdic[cleanfolder]
        except KeyError:
          markettag = ''

    i=i+1
  entityGuid = purviewresults[a]['id']  
  print(entityGuid + ' - ' + clienttag + ' - ' + markettag)

  url = purview_endpoint + "/catalog/api/atlas/v2/entity/guid/"+entityGuid + "/businessmetadata?isOverwrite=true"

  headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
  }
  tagName=''
  if clienttag != '' or markettag != '':
    body = json.dumps(
      {
    "Data Product": {
          "Client": clienttag,
          "Market": markettag
        }
      }
    )
  response = requests.request("POST", url, headers=headers, data=body)
  print(response.text)
print('done')
